In [ ]:
!pip3 install torchsummaryX -q
!pip install datasets==1.18.3 -q
!pip install transformers==4.17.0 -q
!pip install jiwer -q

In [ ]:
import torch
import torchaudio

import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import os
from datasets import Dataset, DatasetDict
import soundfile as sf
import datasets


import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile

import pandas as pd
from tqdm import tqdm
import os
import datetime
print(torch.__version__)
print(torchaudio.__version__)

In [ ]:
!nvidia-smi

In [ ]:
dev_set = datasets.load_dataset('librispeech_asr','clean', split='validation')

In [ ]:
# dev_set = dev_set['validation'].remove_columns(['id', 'speaker_id', 'chapter_id', 'file'])
print(dev_set)
dev_set = dev_set.remove_columns(['id', 'chapter_id', 'file', 'speaker_id'])
def to_lower(example):
  example['text'] = example['text'].lower()
  return example
dev_set = dev_set.map(to_lower)

In [ ]:
root='/home/ubuntu/test/'
def create_dataset(root = root , partition = 'Synthetic ASR- IDL Project/new/'):
  audio_dir = root + partition + "multi-speaker/"
  label_dir = root + partition + "multi-speaker-trans/"

  audio_files = [audio_dir + a for a in sorted(os.listdir(audio_dir))][1:2000]
  label_files = [label_dir + l for l in sorted(os.listdir(label_dir))][1:2000]
  transcripts = []
  for label_file in label_files:
    with open(label_file, "r") as l:
      transcripts.append(l.read().lower())
  audio_dataset = datasets.Dataset.from_dict({
      "audio": audio_files,
      "text": transcripts,
  }).cast_column("audio", datasets.Audio())

  return audio_dataset

dataset = datasets.DatasetDict({
    "train": create_dataset(),
    "test": dev_set

})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict["|"] = vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict[" "] = len(vocab_dict)
vocab_dict

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token=" ", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
import IPython.display as ipd
import numpy as np
import random
timit = dataset
rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=False, rate=16000)

In [ ]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    # print(audio)
    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset,remove_columns=dataset.column_names['train'])

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
# import datasets

wer_metric = datasets.load_metric("wer")

In [ ]:
from transformers import Wav2Vec2ForCTC
from transformers import AutoModel

model = AutoModel.from_pretrained("/home/ubuntu/test/output/new/checkpoint-2000/")

In [ ]:
model.freeze_feature_encoder()

In [ ]:
print(dataset['train'][0]['labels'])


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/home/ubuntu/test/output/new/",
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=5,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer":wer}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=processor.feature_extractor,
)

In [ ]:
gc.collect()
torch.cuda.empty_cache()
trainer.train()

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  print(processor.decode(batch["labels"], group_tokens=False))
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

In [ ]:
print(results)

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))